# Assignment Week 3 - Part 1

### Importing the needed libraries and request the data from the url:

In [117]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)


    
### Creating a soup object and extracting the table part which is relevant

In [99]:
soup = BeautifulSoup(r.text, "html.parser")

table = soup.find("table", attrs = {"class":"wikitable sortable"})
table_data =table.tbody.find_all("tr")
table_data[0:5]

[<tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighbourhood
 </th></tr>,
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>,
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>,
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>]

### Getting the headers of table: 

In [190]:
headings = []
for th in table_data[0].find_all('th'):
    headings.append(th.text.strip())

headings

['Postal Code', 'Borough', 'Neighbourhood']

### Getting the data of the table: 


In [247]:
final_table = pd.DataFrame()

for tr in table_data[1:]:
    #print("This is tr: ", i)
    j = 0
    row = [0,0,0]
    for td in tr.find_all("td"):
        #print("this is td: ", td.text)
        row[j] = td.text.strip()
        j = j+1
    row =  pd.Series(row)
    final_table = final_table.append(row, ignore_index = True)

# renaming the headers 
final_table.rename(columns = {0: headings[0], 1:headings[1], 2:headings[2]}, inplace = True)
final_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Cleaning the data 

In [250]:
#Getting the rows for which the borough is not assigned:
UnassignedBoroughs = final_table[final_table["Borough"]=="Not assigned"].index

#Dropping those rows: 
final_table.drop(UnassignedBoroughs, inplace = True)

#Getting the rows for which the Neighborhood is unassigned and setting those equal to the borough: 
for index, row in final_table.iterrows():
    if (row["Neighbourhood"] == "Not assigned"): 
        row["Neighbourhood"] = row["Borough"]

#Group by the postal code
final_table = final_table.groupby("Postal Code").head()
final_table 

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Shape of the table

In [249]:
print("The shape of the table is: ", final_table.shape)

The shape of the table is:  (103, 3)


# Assignment week 3 - Part 2!

In [253]:
import geocoder
import wget

Getting the csv file!

In [262]:
#Downloading the file 
url = "http://cocl.us/Geospatial_data"
wget.download(url, "geospatial_coordinates.csv")

'geospatial_coordinates.csv'

In [264]:
#From csv file to pandas dataframe:
Geospatial_data = pd.read_csv('geospatial_coordinates.csv', header = 0)
Geospatial_data.head()

final_table = final_table.merge(Geospatial_data, how = "left", left_on = "Postal Code", right_on = "Postal Code")
final_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Assignment Week 3 - Part 3 - Exploration and clustering of the data!

In [278]:
# Import needed libraries: 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json # library to handle JSON files
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


Creating a map of Toronto with the neigborhood superimposed: 

In [276]:
# create map of Toronto using latitude and longitude values
latitude = 43.651070
longitude =-79.347015
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_table['Latitude'], final_table['Longitude'], final_table['Borough'], final_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [277]:
# Foursquare credentials 
CLIENT_ID = '******************' # your Foursquare ID
CLIENT_SECRET = '*********************' # your Foursquare Secret
ACCESS_TOKEN = '**************************' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3P1TUXWEL0M0QHRJZXJK2E224H31PMMMUV3A0XPLJIX24HY0
CLIENT_SECRET:MPSRTTAZXMJ0VITA2RODBQ0CEEMMR3JF00AJIUL2DLXGMKVA


In [279]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [280]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [298]:
Toronto_venues = getNearbyVenues(names=final_table['Neighbourhood'],
                                   latitudes=final_table['Latitude'],
                                   longitudes=final_table['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [303]:
# Checking out the venues in toronto:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [299]:
# Grouping by neighborhood
Toronto_venues.groupby(["Venue Category","Neighborhood"]).count().head(10)

Neighborhood Latitude  \
Venue Category      Neighborhood                                                                
Accessories Store   Lawrence Manor, Lawrence Heights                                        1   
Airport             CN Tower, King and Spadina, Railway Lands, Harb...                      1   
                    Downsview                                                               1   
Airport Food Court  CN Tower, King and Spadina, Railway Lands, Harb...                      1   
Airport Gate        CN Tower, King and Spadina, Railway Lands, Harb...                      1   
Airport Lounge      CN Tower, King and Spadina, Railway Lands, Harb...                      2   
Airport Service     CN Tower, King and Spadina, Railway Lands, Harb...                      3   
Airport Terminal    CN Tower, King and Spadina, Railway Lands, Harb...                      2   
American Restaurant Bedford Park, Lawrence Manor East                                       1   
                    Canada Post Gateway Processing Centre                                   1   

                                                                        Neighborhood Longitude  \
Venue Category      Neighborhood                                                                 
Accessories Store   Lawrence Manor, Lawrence Heights                                         1   
Airport             CN Tower, King and Spadina, Railway Lands, Harb...                       1   
                    Downsview                                                                1   
Airport Food Court  CN Tower, King and Spadina, Railway Lands, Harb...                       1   
Airport Gate        CN Tower, King and Spadina, Railway Lands, Harb...                       1   
Airport Lounge      CN Tower, King and Spadina, Railway Lands, Harb...                       2   
Airport Service     CN Tower, King and Spadina, Railway Lands, Harb...                       3   
Airport Terminal    CN Tower, King and Spadina, Railway Lands, Harb...                       2   
American Restaurant Bedford Park, Lawrence Manor East                                        1   
                    Canada Post Gateway Processing Centre                                    1   

                                                                        Venue  \
Venue Category      Neighborhood                                                
Accessories Store   Lawrence Manor, Lawrence Heights                        1   
Airport             CN Tower, King and Spadina, Railway Lands, Harb...      1   
                    Downsview                                               1   
Airport Food Court  CN Tower, King and Spadina, Railway Lands, Harb...      1   
Airport Gate        CN Tower, King and Spadina, Railway Lands, Harb...      1   
Airport Lounge      CN Tower, King and Spadina, Railway Lands, Harb...      2   
Airport Service     CN Tower, King and Spadina, Railway Lands, Harb...      3   
Airport Terminal    CN Tower, King and Spadina, Railway Lands, Harb...      2   
American Restaurant Bedford Park, Lawrence Manor East                       1   
                    Canada Post Gateway Processing Centre                   1   

                                                                        Venue Latitude  \
Venue Category      Neighborhood                                                         
Accessories Store   Lawrence Manor, Lawrence Heights                                 1   
Airport             CN Tower, King and Spadina, Railway Lands, Harb...               1   
                    Downsview                                                        1   
Airport Food Court  CN Tower, King and Spadina, Railway Lands, Harb...               1   
Airport Gate        CN Tower, King and Spadina, Railway Lands, Harb...               1   
Airport Lounge      CN Tower, King and Spadina, Railway Lands, Harb...               2   
Airport Service     CN Tower, King an

Create one-hot encoding

In [304]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping by neighborhood and calculating frequencies 

In [305]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.038462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [306]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [308]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Pub,Dance Studio,Coffee Shop,Gym,Gastropub,Coworking Space,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Bridal Shop,Sandwich Place,Deli / Bodega,Restaurant,Intersection,Supermarket
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Indian Restaurant,Restaurant,Pub,Pizza Place,Pharmacy,Locksmith,Liquor Store


Cluster neighborhoods: 

In [331]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1,
       1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 4, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 7, 0, 1, 1, 1, 1, 1, 1, 4, 1, 1, 0, 1, 1, 1, 0, 1,
       8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 0,
       1, 0, 1, 1, 1, 1, 6])

In [329]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = neighborhoods_venues_sorted.merge(right = final_table[["Neighbourhood", "Latitude", "Longitude"]], how = "left", left_on = "Neighborhood", right_on = "Neighbourhood")
toronto_merged.drop("Neighbourhood", inplace = True, axis = 1)
toronto_merged

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,43.794200,-79.262029
1,1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Athletics & Sports,Pub,Dance Studio,Coffee Shop,Gym,Gastropub,Coworking Space,Distribution Center,43.602414,-79.543484
2,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Bridal Shop,Sandwich Place,Deli / Bodega,Restaurant,Intersection,Supermarket,43.754328,-79.442259
3,1,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,43.786947,-79.385975
4,1,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Indian Restaurant,Restaurant,Pub,Pizza Place,Pharmacy,Locksmith,Liquor Store,43.733283,-79.419750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1,"Willowdale, Willowdale East",Ramen Restaurant,Sandwich Place,Coffee Shop,Café,Electronics Store,Sushi Restaurant,Steakhouse,Bank,Ice Cream Shop,Lounge,43.770120,-79.408493
95,1,"Willowdale, Willowdale West",Grocery Store,Coffee Shop,Discount Store,Pharmacy,Pizza Place,Airport Gate,Airport Lounge,Electronics Store,Eastern European Restaurant,Drugstore,43.782736,-79.442259
96,1,Woburn,Coffee Shop,Indian Restaurant,Korean BBQ Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,43.770992,-79.216917
97,1,Woodbine Heights,Skating Rink,Park,Curling Ice,Beer Store,Intersection,Bus Stop,Athletics & Sports,Construction & Landscaping,Comfort Food Restaurant,Eastern European Restaurant,43.695344,-79.318389


In [330]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters